In [ ]:
TAMBO_PATH = "../Tambo/"
# This should be the numpy file with the indicess of the triggered events
TRIGGERED_EVENTS_FILE = ""
# This should be the simulation parameters jld2 file
SIMULATION_FILE = "./output/Oct16th2023_WhitePaper_300k.jld2"

In [ ]:
using Pkg
Pkg.activate(TAMBO_PATH)
using Tambo
using PyCall
using JLD2

In [ ]:
np = pyimport("numpy")

In [ ]:
sim = jldopen(SIMULATION_FILE)
config = SimulationConfig(; geo_spline_path="../resources/tambo_spline.jld2", filter(x->x[1]!=:geo_spline_path, sim["config"])...)
geo = Tambo.Geometry(config)
injector = Tambo.Injector(config);

In [ ]:
triggered_events = rand(1:length(sim["injected_events"]), 500);
# You should uncomment this
# triggered_events = np.load(TRIGGERED_EVENTS_FILE);

In [ ]:
# Global fit results from ArXiv:2308.00191
γ = 2.52
norm = 1.8e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
pl = Tambo.PowerLaw(γ, 100units.GeV, 1e9units.GeV, norm)

In [ ]:
events = sim["injected_events"][triggered_events]

fluxes = pl.(events["initial_state"]["energy"])
weights = oneweight.(events, Ref(injector), Ref(injector.xs)) ./ maximum(triggered_events)

rates = fluxes .* weights
println("Number of events in 3 J-years: $(sum(rates) * 3 * 10^7.5 * units.second)")